## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-16-54-20 +0000,nyt,Zelensky’s Image Is Stained as Corruption Inqu...,https://www.nytimes.com/2025/11/14/world/europ...
1,2025-11-14-16-53-58 +0000,bbc,Man guilty of stalking Dame Penny Mordaunt,https://www.bbc.com/news/articles/c4gjl768g8zo...
2,2025-11-14-16-51-29 +0000,nyt,Rapist Called One of UK’s Worst Sex Offenders ...,https://www.nytimes.com/2025/11/14/world/europ...
3,2025-11-14-16-50-00 +0000,wsj,"Gold Slides Amid Selloff, Lower December Fed R...",https://www.wsj.com/finance/commodities-future...
4,2025-11-14-16-42-56 +0000,nyt,Indigenous People Take the Stage at COP30 Clim...,https://www.nytimes.com/2025/11/14/climate/cop...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,36
63,new,20
177,california,16
227,market,11
29,cut,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-11-14-15-55-00 +0000,wsj,New Prosecutor Named to Oversee Georgia Crimin...,https://www.wsj.com/us-news/law/new-prosecutor...,82
340,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,78
50,2025-11-14-15-16-18 +0000,nypost,Prosecutor appoints himself to take over Georg...,https://nypost.com/2025/11/14/us-news/prosecut...,77
282,2025-11-13-21-21-08 +0000,nypost,BBC apologizes to Trump over selectively edite...,https://nypost.com/2025/11/13/us-news/bbc-apol...,76
202,2025-11-14-03-04-50 +0000,nypost,Va. prosecutor accused of ‘stymying the invest...,https://nypost.com/2025/11/13/us-news/va-prose...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,82,2025-11-14-15-55-00 +0000,wsj,New Prosecutor Named to Oversee Georgia Crimin...,https://www.wsj.com/us-news/law/new-prosecutor...
288,54,2025-11-13-20-54-33 +0000,latimes,Starbucks union launches Red Cup Day strike at...,https://www.latimes.com/business/story/2025-11...
277,46,2025-11-13-21-46-58 +0000,nypost,Stocks have worst day in a month amid AI conce...,https://nypost.com/2025/11/13/business/stocks-...
199,45,2025-11-14-03-37-55 +0000,nypost,NYC Mayor-elect Zohran Mamdani holds secretive...,https://nypost.com/2025/11/13/us-news/nyc-mayo...
282,40,2025-11-13-21-21-08 +0000,nypost,BBC apologizes to Trump over selectively edite...,https://nypost.com/2025/11/13/us-news/bbc-apol...
30,39,2025-11-14-16-00-00 +0000,wsj,"Our compulsive purchasing of cheap, foreign-ma...",https://www.wsj.com/economy/consumers/will-tru...
340,36,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
10,35,2025-11-14-16-31-31 +0000,nypost,Trump asks DOJ to probe Jeffrey Epstein ties t...,https://nypost.com/2025/11/14/us-news/trump-as...
68,34,2025-11-14-14-25-43 +0000,nypost,Mom of 5-year-old boy killed by his accused dr...,https://nypost.com/2025/11/14/us-news/mom-of-5...
337,32,2025-11-13-17-22-19 +0000,nyt,Israeli Settlers Burn West Bank Mosque Amid Ri...,https://www.nytimes.com/2025/11/13/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
